In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from Utils.build_dataset import paths_dataset
from torchvision import transforms
from Models.FCN import FCN
from Utils.FishDataset import FishDataset
from Utils.training import training_loop, predict, iou_np
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import cv2
import os

In [ ]:
dtype = torch.float32
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
print('using device:', device)

In [ ]:
label_dict = {0 : "Black Sea Sprat", 1 : "Sea Bass", 2 : "Red Mullet", 3 : "Trout", 4 : "Striped Red Mullet", 
              5 : "Shrimp", 6 : "Red Sea Bream", 7 : "Hourse Mackerel", 8 : "Gilt-Head Bream"}

## Carregando as Imagens

In [ ]:
path = 'Dataset/Fish_Dataset/Fish_Dataset/'
df_train_pd, df_valid_pd, df_test_pd = paths_dataset(path)

In [ ]:
img_size = 256
threshold = 0.5
transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize((img_size, img_size)), transforms.ToTensor(), 
                               transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
transform_mask = transforms.Compose([transforms.ToPILImage(), transforms.Resize((img_size, img_size)), transforms.ToTensor()])

In [ ]:
df_train = FishDataset(df_train_pd, transform, transform_mask)

In [ ]:
df_valid = FishDataset(df_valid_pd, transform, transform_mask)

In [ ]:
df_test = FishDataset(df_test_pd, transform, transform_mask)

In [ ]:
df_train[0][1].shape

In [ ]:
for i in range(3):
    n = random.randint(0, (len(df_train)))
    fig = plt.figure(figsize=(10, 100))
    
    fig.add_subplot(15, 2, 1)
    #plt.title(label_dict[df_train[n][2]])
    plt.imshow(df_train[n][0].permute(1, 2, 0))
    
    fig.add_subplot(15, 2, 2)
    #plt.title(label_dict[df_train[n][2]])
    plt.imshow(df_train[n][1].permute(1, 2, 0))

## Construindo o Modelo

In [ ]:
epochs = 20
batch_size = 32
lr = 1e-3
train_loader = DataLoader(df_train, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(df_valid, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(df_test, batch_size=batch_size, shuffle=False, num_workers=4)
criterion = torch.nn.BCEWithLogitsLoss()
sigmoid = torch.nn.Sigmoid()

In [ ]:
len(train_loader.dataset)

In [ ]:
train_losses_list = []
train_loss_final = []
train_iou_list = []
train_iou_final = []
valid_losses_list = []
valid_loss_final = []
valid_iou_list = []
valid_iou_final = []
test_iou_list = []
test_acc_list = []

In [ ]:
img_list_test = list(df_test_pd['image'])
mask_list_test = list(df_test_pd['mask'])
label_list_test = list(df_test_pd['label'])
label_id_list_test = list(df_test_pd['label_id'])

### Configuração dos Experimentos

In [ ]:
exp_number = 1 #id do experimento, será usado para salvar o modelo com nome único
n_exps = 30 #Quantidade de experimentos que serão executados
path_pkl = 'Saved Models/FCN Segmentation' #Path da pasta onde serão salvos os arquivos pkl
path_metrics = 'Metrics' #Path da pasta onde será salvo o arquivo csv com as métricas

In [ ]:
for n in range(exp_number, (n_exps)):
    print("Run {0} out of {1}".format(n, n_exps))
    model = FCN()
    model.to(device, dtype=dtype)
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    model, optimizer, train_losses, valid_losses, train_iou, valid_iou = training_loop(epochs, model, train_loader, 
                                                                                       valid_loader, criterion, optimizer, 
                                                                                       device, dtype)
    train_losses_list.append(train_losses)
    train_iou_list.append(train_iou)
    valid_losses_list.append(valid_losses)
    valid_iou_list.append(valid_iou)
    train_loss_final.append(train_losses[-1])
    train_iou_final.append(train_iou[-1])
    valid_loss_final.append(valid_losses[-1])
    valid_iou_final.append(valid_iou[-1])
    
    save_name = f'{path_pkl}/fcn_segmentation_run_{n}.pkl'
    torch.save(model.state_dict(), save_name)
    print(f'Model saved in {save_name}')
    
    running_iou = 0
    for i in range(len(img_list_test)):
        label_id = label_id_list_test[i]
        image = cv2.imread(img_list_test[i])
        image = cv2.resize(image, (img_size, img_size))
        mask = cv2.imread(mask_list_test[i])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = cv2.resize(mask, (img_size, img_size))
        pred = predict(model, image, img_size, threshold, device)
        iou_pred = iou_np(mask, pred)
        running_iou += iou_pred
        

    iou_test = (running_iou / len(img_list_test)) * 100
    test_iou_list.append(iou_test)

## Salvando as Métricas dos experimentos

In [ ]:
metrics = {'train_loss_list': train_losses_list,
          'train_loss': train_loss_final,
          'train_iou_list' : train_iou_list,
          'train_iou': train_iou_final,
          'valid_losses_list': valid_losses_list,
          'valid_loss': valid_loss_final,
          'valid_iou_list': valid_iou_list,
          'valid_iou': valid_iou_final,
          'test_iou': test_iou_list}

In [ ]:
df_metrics = pd.DataFrame(metrics)

In [ ]:
df_metrics.shape

In [ ]:
path_csv = f'{path_metrics}/metrics_fcn_segmentation.csv'
df_metrics.to_csv(path_csv, sep='\t')